

Você foi contratado por uma empresa de pesquisa de mercado interessada em automatizar a análise de sentimentos de comentários de clientes de diferentes setores (filmes, produtos, restaurantes). Para isso, você utilizará o dataset Sentiment Labelled Sentences ([clique aqui](https://archive.ics.uci.edu/dataset/331/sentiment+labelled+sentences)), que contém comentários já rotulados como positivos ou negativos.

O objetivo é criar um modelo de classificação de sentimentos que seja capaz de analisar os sentimentos expressos nesses comentários e fornecer insights rápidos para a empresa.

Este dataset contém comentários retirados de três fontes diferentes: Yelp, Amazon e IMDb, com cada linha sendo rotulada como “0” (negativo) ou “1” (positivo).


**Tarefas:**

1. Quais bibliotecas e ferramentas são necessárias para realizar essa análise? Importe essas bibliotecas em um ambiente de desenvolvimento como Google Colab ou Jupyter Notebook.

2. Como podemos transformar os textos brutos em uma forma utilizável para o modelo? Utilize a técnica TfidfVectorizer para converter os textos em uma matriz de características numéricas. Explique por que o pré-processamento é essencial para a análise de texto.

3. Qual modelo de classificação devemos usar e por quê? Construa um pipeline que integra o pré-processamento de texto e o modelo. Explique a sua escolha de modelo para a tarefa de análise de sentimento.

4. Como treinamos o modelo para que ele aprenda a classificar os comentários? Treine o modelo Naive Bayes usando os dados de treino. Descreva o processo de treinamento e como o modelo aprende a partir dos exemplos rotulados.

5. Como podemos avaliar o desempenho do modelo? Avalie o modelo usando o conjunto de teste e calcule a acurácia. Explique o que a acurácia nos diz sobre a qualidade das previsões do modelo.

Boa prática!


### 1. Bibliotecas

In [25]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


### 2. Carregamento dos Dados e Pré-processamento

##### Explicação:

O pré-processamento é essencial pois textos brutos contêm ruído (stopwords, pontuação, etc.), e modelos matemáticos só processam números. O TF-IDF converte textos em vetores numéricos, removendo stopwords (como 'the', 'and'),
ignorando pontuações e convertendo para minúsculas. Além disso, atribui pesos às palavras com base em sua frequência (TF) e relevância no corpus (IDF), destacando termos importantes e reduzindo dimensionalidade.

In [26]:
# Carregar os arquivos (ajuste os nomes conforme seus arquivos)
files = {
    'yelp': 'yelp_labelled.txt',
    'amazon': 'amazon_cells_labelled.txt',
    'imdb': 'imdb_labelled.txt'
}

dfs = []
for source, path in files.items():
    df = pd.read_csv(path, sep='\t', header=None, names=['text', 'label'])
    df['source'] = source  # Adicionar coluna para identificar a origem
    dfs.append(df)

# Combinar todos os dados em um único DataFrame
data = pd.concat(dfs, ignore_index=True)

# Pré-processamento
# Separar features (texto) e target (label)
X = data['text']
y = data['label']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar pipeline com TF-IDF
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


### 3. Modelo de classificação

#### Explicação:
O Naive Bayes é ideal para texto por:
1. Eficiência: Treina rapidamente mesmo com milhares de palavras.
2. Dados esparsos: Funciona bem quando a maioria dos valores é zero (como em matrizes TF-IDF).
3. Independência entre palavras: Assume que a presença de uma palavra não afeta outra, simplificando cálculos e funcionando bem na prática para classificação binária.



In [27]:
# Criar pipeline com TF-IDF e Naive Bayes
modelo = make_pipeline(
    TfidfVectorizer(max_features=10000, stop_words='english'),  # Limita a 10000 palavras
    MultinomialNB()
)


### 4. Treinamento

#### Explicação:
Durante o treinamento, o Naive Bayes calcula:
1. Probabilidades a priori: Percentual de textos positivos/negativos no treino.
2. Probabilidades condicionais: Para cada palavra, calcula a chance de aparecer em textos positivos vs. negativos (ex: a palavra 'ótimo' tem alta probabilidade na classe positiva).


In [28]:
modelo.fit(X_train, y_train)


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=10000, stop_words='english')),
                ('multinomialnb', MultinomialNB())])

### 5. Avaliação do Modelo

#### Explicação:

A acurácia de 76.73% indica que o modelo classifica corretamente ~77% dos comentários. O resultado já é útil para análises preliminares, mas ainda comete erros significativos e pode ser melhorado.

In [29]:
# Previsões e acurácia
y_pred = modelo.predict(X_test)
acuracia = accuracy_score(y_test, y_pred)
print(f"Acurácia: {acuracia * 100:.2f}%")


Acurácia: 76.73%
